#Build global stats

##Import data and libraries

In [2]:
import iofunctions as iof
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import numpy as np
import seaborn as sns
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

#defining 10-color pallette using http://flatui.com/flat-ui-color-palette/
#light
flatui_l = ["#1abc9c", "#2ecc71", "#3498db", "#9b59b6", "#34495e", "#f1c40f", "#e67e22", "#e74c3c", "#ecf0f1", "#95a5a6"]
#dark
flatui_d = ["#16a085", "#27ae60", "#2980b9", "#8e44ad", "#2c3e50", "#f39c12", "#d35400", "#c0392b", "#bdc3c7", "#7f8c8d"]

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

#importing global data
mergedSpineDFMetaCompleteID = pd.read_excel('output/mergedSpineDFMetaCompleteID.xlsx','Sheet1')

#separating measures for Odd and Normal
N = mergedSpineDFMetaCompleteID[mergedSpineDFMetaCompleteID['oddType'] == 'Normal']
C = mergedSpineDFMetaCompleteID

#prefiltering basal and apical dendrites of l5
Napical = N[(N.dendriteType=="apical")&(N.pyramidalLayer=="l5")]
Nbasal = N[(N.dendriteType=="basal")&(N.pyramidalLayer=="l5")]
#prefiltering basal and apical dendrites of l5
Capical = C[(C.dendriteType=="apical")&(C.pyramidalLayer=="l5")]
Cbasal = C[(C.dendriteType=="basal")&(C.pyramidalLayer=="l5")]

##create pivot table with grouping by MouseID for each measurement

###data groupings

In [3]:
CaSD = Capical.groupby(['Genotype','mouseID'], sort=True)['2D Spine Density 1/um']
NaSL = Napical.groupby(['Genotype','mouseID'], sort=True)["Spine Length, um"]
NaHW = Napical.groupby(['Genotype','mouseID'], sort=True)["Head Width, um"]
NaST = Napical.groupby(['Genotype','mouseID'], sort=True)["Straightness"]

CbSD = Capical.groupby(['Genotype','mouseID'], sort=True)['2D Spine Density 1/um']
NbSL = Nbasal.groupby(['Genotype','mouseID'], sort=True)["Spine Length, um"]
NbHW = Nbasal.groupby(['Genotype','mouseID'], sort=True)["Head Width, um"]
NbST = Nbasal.groupby(['Genotype','mouseID'], sort=True)["Straightness"]

###plotting for each mouse and genotype

In [4]:
#introducing Standard Error of the Mean function
sem = lambda x: (x.std() / np.sqrt(x.count()))
#colors of flatui_d
color8 = sns.color_palette(flatui_d,8)

#ApicalL5['Spine Length, um'].plot.hist(stacked=False, bins=30)
fig, axes = plt.subplots(nrows=2, ncols=4, sharey=True, figsize=(12, 6))

#Apical
ax = CaSD.mean().plot.barh(xerr=CaSD.apply(sem), color=color8, ax = axes[0,0])
ax.set_title('2D Spine Density 1/um')
ax.set_ylabel('Apical')

ax = NaSL.mean().plot.barh(xerr=NaSL.apply(sem), color=color8, ax = axes[0,1])
ax.set_title("Spine Length, um")

ax = NaHW.mean().plot.barh(xerr=NaHW.apply(sem), color=color8, ax = axes[0,2])
ax.set_title("Head Width, um")

ax = NaST.mean().plot.barh(xerr=NaST.apply(sem), color=color8, ax = axes[0,3])
ax.set_xlim([0.95,1])
ax.set_title("Straightness")

#Basal
ax = CbSD.mean().plot.barh(xerr=CbSD.apply(sem), color=color8, ax = axes[1,0])
ax.set_ylabel('Basal')

ax = NbSL.mean().plot.barh(xerr=NbSL.apply(sem), color=color8, ax = axes[1,1])

ax = NbHW.mean().plot.barh(xerr=NbHW.apply(sem), color=color8, ax = axes[1,2])

ax = NbST.mean().plot.barh(xerr=NbST.apply(sem), color=color8, ax = axes[1,3])
ax.set_xlim([0.95,1])

plt.savefig('mouseID-stats.png')

##WT vs. CKO

###plot differences

In [7]:
fig, axes = plt.subplots(nrows=2, ncols=4, sharex=True, figsize=(10, 8))

#Apical
ax = sns.boxplot(y="2D Spine Density 1/um", x="Genotype", data=Capical, palette="Set3", width=0.4, ax = axes[0,0])
ax.set_title('2D Spine Density 1/um')
ax.set_ylabel('Apical')
ax = sns.boxplot(y="Spine Length, um", x="Genotype", data=Napical, palette="Set3", width=0.4, ax = axes[0,1])
ax.set_title("Spine Length, um")
ax = sns.boxplot(y="Head Width, um", x="Genotype", data=Napical, palette="Set3", width=0.4, ax = axes[0,2])
ax.set_title("Head Width, um")
ax = sns.boxplot(y="Straightness", x="Genotype", data=Napical, palette="Set3", width=0.4, ax = axes[0,3])
ax.set_title("Straightness")

#Basal
ax = sns.boxplot(y="2D Spine Density 1/um", x="Genotype", data=Cbasal, palette="Set3", width=0.4, ax = axes[1,0])
ax.set_ylabel('Basal')
ax = sns.boxplot(y="Spine Length, um", x="Genotype", data=Nbasal, palette="Set3", width=0.4, ax = axes[1,1])
ax = sns.boxplot(y="Head Width, um", x="Genotype", data=Nbasal, palette="Set3", width=0.4, ax = axes[1,2])
ax = sns.boxplot(y="Straightness", x="Genotype", data=Nbasal, palette="Set3", width=0.4, ax = axes[1,3])
plt.savefig('WT-CKO.png')

###set data for comparison 2D spine density

In [3]:
#l5
apicalCKOl5SD = Capical[Capical.Genotype=="CKO"]['2D Spine Density 1/um'].drop_duplicates()
apicalWTl5SD = Capical[Capical.Genotype=="WT"]['2D Spine Density 1/um'].drop_duplicates()

basalCKOl5SD = Cbasal[Cbasal.Genotype=="CKO"]['2D Spine Density 1/um'].drop_duplicates()
basalWTl5SD = Cbasal[Cbasal.Genotype=="WT"]['2D Spine Density 1/um'].drop_duplicates()

#l3, not using forward due to low amounts of data
apicalCKOl3SD = C[(C.Genotype=="CKO") & (C.dendriteType=="apical")&(C.pyramidalLayer=="l3")]['2D Spine Density 1/um'].drop_duplicates()
apicalWTl3SD = C[(C.Genotype=="WT") & (C.dendriteType=="apical")&(C.pyramidalLayer=="l3")]['2D Spine Density 1/um'].drop_duplicates()

basalCKOl3SD = C[(C.Genotype=="CKO") & (C.dendriteType=="basal")&(C.pyramidalLayer=="l3")]['2D Spine Density 1/um'].drop_duplicates()
basalWTl3SD = C[(C.Genotype=="WT") & (C.dendriteType=="basal")&(C.pyramidalLayer=="l3")]['2D Spine Density 1/um'].drop_duplicates()

###run similarity tests, calculate p-values for 2D spine density on l5

In [4]:
import numpy as np
from scipy.stats import ttest_1samp, wilcoxon, ttest_ind, mannwhitneyu
from scipy.special import stdtr

print("Apical Dendrites on l5===============")
t, p = ttest_ind(apicalCKOl5SD, apicalWTl5SD, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(apicalCKOl5SD, apicalWTl5SD)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

print("Basal Dendrites on l5================")
t, p = ttest_ind(basalCKOl5SD, basalWTl5SD, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(basalCKOl5SD, basalWTl5SD)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

Apical Dendrites on l5===============
Two-sample t-test:                        t = -5.13762  p = 1.06513e-06
Two-sample Wilcoxon Mann Whitney test,    p= 3.15896011998e-06
Basal Dendrites on l5================
Two-sample t-test:                        t = -4.26078  p = 4.372e-05
Two-sample Wilcoxon Mann Whitney test,    p= 5.23678296759e-07


###publish describe stats

In [5]:
print("WT Apical===============")
print(apicalWTl5SD.describe())
print("CKO Apical==============")
print(apicalCKOl5SD.describe())

print("WT Basal===============")
print(basalWTl5SD.describe())
print("CKO Basal==============")
print(basalCKOl5SD.describe())

WT Apical===============
count    103.000000
mean       0.453472
std        0.124546
min        0.243126
25%        0.361633
50%        0.425973
75%        0.529505
max        0.801723
Name: 2D Spine Density 1/um, dtype: float64
CKO Apical==============
count    52.000000
mean      0.357452
std       0.101649
min       0.178879
25%       0.286610
50%       0.341836
75%       0.423054
max       0.628959
Name: 2D Spine Density 1/um, dtype: float64
WT Basal===============
count    71.000000
mean      0.383170
std       0.114646
min       0.172748
25%       0.287840
50%       0.365929
75%       0.469171
max       0.704303
Name: 2D Spine Density 1/um, dtype: float64
CKO Basal==============
count    56.000000
mean      0.286865
std       0.135063
min       0.067741
25%       0.211596
50%       0.251255
75%       0.326263
max       0.831316
Name: 2D Spine Density 1/um, dtype: float64


### Other Stats

In [22]:
mergedSpineDFMetaNormalApical = mergedSpineDFMetaComplete[(mergedSpineDFMetaComplete['oddType'] == "Normal")&(mergedSpineDFMetaComplete.dendriteType=="apical")&(mergedSpineDFMetaComplete.pyramidalLayer=="l5")].drop_duplicates()
mergedSpineDFMetaNormalBasal = mergedSpineDFMetaComplete[(mergedSpineDFMetaComplete['oddType'] == "Normal")&(mergedSpineDFMetaComplete.dendriteType=="basal")&(mergedSpineDFMetaComplete.pyramidalLayer=="l5")].drop_duplicates()

#"Spine Length, um"
#"Head Width, um"
#"Straightness"
CKOa_sl=Napical[Napical.Genotype=="CKO"]["Spine Length, um"]
WTa_sl=Napical[Napical.Genotype=="WT"]["Spine Length, um"]
CKOa_hw=Napical[Napical.Genotype=="CKO"]["Head Width, um"]
WTa_hw=Napical[Napical.Genotype=="WT"]["Head Width, um"]
CKOa_st=Napical[Napical.Genotype=="CKO"]["Straightness"]
WTa_st=Napical[Napical.Genotype=="WT"]["Straightness"]

CKOb_sl=Napical[Napical.Genotype=="CKO"]["Spine Length, um"]
WTb_sl=Napical[Napical.Genotype=="WT"]["Spine Length, um"]
CKOb_hw=Napical[Napical.Genotype=="CKO"]["Head Width, um"]
WTb_hw=Napical[Napical.Genotype=="WT"]["Head Width, um"]
CKOb_st=Napical[Napical.Genotype=="CKO"]["Straightness"]
WTb_st=Napical[Napical.Genotype=="WT"]["Straightness"]

###run similarity tests, calculate p-values

In [23]:
print("Apical Spine Length on l5===============")
t, p = ttest_ind(CKOa_sl, WTa_sl, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOa_sl, WTa_sl)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

print("Basal Spine Length on l5================")
t, p = ttest_ind(CKOb_sl, WTb_sl, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOb_sl, WTb_sl)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)
print("------------------------------------------------------------")

print("Apical Head Width on l5===============")
t, p = ttest_ind(CKOa_hw, WTa_hw, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOa_hw, WTa_hw)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

print("Basal Head Width on l5================")
t, p = ttest_ind(CKOb_hw, WTb_hw, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOb_hw, WTb_hw)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)
print("------------------------------------------------------------")

print("Apical Straightness on l5===============")
t, p = ttest_ind(CKOa_st, WTa_st, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOa_st, WTa_st)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

print("Basal Straightness on l5================")
t, p = ttest_ind(CKOb_st, WTb_st, equal_var=False)
print("Two-sample t-test:                        t = %g  p = %g" % (t, p))
u, p_value = mannwhitneyu(CKOb_st, WTb_st)
print("Two-sample Wilcoxon Mann Whitney test,    p=", p_value)

Apical Spine Length on l5===============
Two-sample t-test:                        t = 1.49322  p = 0.135558
Two-sample Wilcoxon Mann Whitney test,    p= 0.240619911428
Basal Spine Length on l5================
Two-sample t-test:                        t = 2.62353  p = 0.00881029
Two-sample Wilcoxon Mann Whitney test,    p= 0.0348179803957
------------------------------------------------------------
Apical Head Width on l5===============
Two-sample t-test:                        t = -0.11652  p = 0.907254
Two-sample Wilcoxon Mann Whitney test,    p= 0.0616287736056
Basal Head Width on l5================
Two-sample t-test:                        t = 1.84276  p = 0.0655972
Two-sample Wilcoxon Mann Whitney test,    p= 0.357395660809
------------------------------------------------------------
Apical Straightness on l5===============
Two-sample t-test:                        t = -1.78907  p = 0.0737889
Two-sample Wilcoxon Mann Whitney test,    p= 0.00507003676406
Basal Straightness on l5===